In [18]:
# dependencies
import pandas as pd
import psycopg2 as pg
from datetime import date
from configparser import ConfigParser

In [19]:
# reading the configuration file containing the postgres credentials
config = ConfigParser()
config.read("pg_creds.cfg")

[]

In [10]:
#############################################################################
# Extract / Transform
#############################################################################


def fetchDataToLocal():
    """
    we use the python requests library to fetch the nyc in json format, then
    use the pandas library to easily convert from json to a csv saved in the
    local data directory
    """
    
    # fetching the request
    df = pd.read_csv("https://data.stadt-zuerich.ch/dataset/ugz_meteodaten_tagesmittelwerte/download/ugz_ogd_meteo_d1_2022.csv",index_col=None)
    
    # for integrity reasons, let's attach the current date to the filename
    df.to_csv("ugz_ogd_meteo_d1_2022_{}.csv".format(date.today().strftime("%Y%m%d")))
    

#############################################################################
# Load
#############################################################################


def sqlLoad():
    """
    we make the connection to postgres using the psycopg2 library, create
    the schema to hold our covid data, and insert from the local csv file
    """
    
    # attempt the connection to postgres
    try:
        dbconnect = pg.connect(
            database=config.get("postgres", "DATABASE"),
            user=config.get("postgres", "USERNAME"),
            password=config.get("postgres", "PASSWORD"),
            host=config.get("postgres", "HOST")
        )
    except Exception as error:
        print(error)
    
    # create the table if it does not already exist
    cursor = dbconnect.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather_data (
            Datum DATE,
            Standort CHAR(255),
            Parameter CHAR(255),
            Intervall CHAR(255),
            Einheit CHAR(255),
            Wert INT,
            Status CHAR(255)
        );
        
        TRUNCATE TABLE weather_data;
    """
    )
    dbconnect.commit()
    
    # insert each csv row as a record in our database
    with open("ugz_ogd_meteo_d1_2022_{}.csv".format(date.today().strftime("%Y%m%d"))) as f:
        next(f) # skip the first row (header)
        for row in f:
            cursor.execute("""
                INSERT INTO covid_data
                VALUES ('{}', '{}', '{}', '{}')
            """.format(
            row.split(",")[0],
            row.split(",")[1],
            row.split(",")[2],
            row.split(",")[3])
            )
    dbconnect.commit()

                   Datum                 Standort Parameter Intervall Einheit  \
0  2022-01-01T00:00+0100  Zch_Stampfenbachstrasse         T        d1      °C   
1  2022-01-01T00:00+0100  Zch_Stampfenbachstrasse   RainDur        d1     min   
2  2022-01-01T00:00+0100  Zch_Stampfenbachstrasse         p        d1     hPa   
3  2022-01-01T00:00+0100  Zch_Stampfenbachstrasse    StrGlo        d1    W/m2   
4  2022-01-01T00:00+0100  Zch_Stampfenbachstrasse  T_max_h1        d1      °C   

     Wert        Status  
0    6.77  provisorisch  
1    0.00  provisorisch  
2  976.43  provisorisch  
3   66.36  provisorisch  
4   10.31  provisorisch  
